In [1]:
import tiledbsoma.io
from tiledbsoma import Experiment
from tiledbsoma.io.outgest import _extract_uns
from pandas import DataFrame as DF
import pandas as pd
from anndata import AnnData
import numpy as np
from os.path import exists
from shutil import rmtree

## Make a simple AnnData

In [2]:
obs = DF({'obs_col': list("ABC")}, index=list("abc"))
var = DF({'var_col': list("XY")}, index=list("xy"))
uns_df0 = DF({'uns_col': list("ghijkl")})
adata = AnnData(
    obs=obs,
    var=var,
    X=np.zeros([len(obs), len(var)]),
    uns={ 'df': uns_df0.copy() },
)
adata

AnnData object with n_obs × n_vars = 3 × 2
    obs: 'obs_col'
    var: 'var_col'
    uns: 'df'

## Ingest to SOMA

In [3]:
path = 'exp'
if exists(path):
    rmtree(path)
tiledbsoma.io.from_anndata(path, adata, measurement_name="RNA")
exp = Experiment.open(path)
uns = exp.ms["RNA"]['uns']
uns

/Users/ryan/.pyenv/versions/3.11.6/lib/python3.11/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)


<Collection 'file:///Users/ryan/c/tiledb/repros/exp/ms/RNA/uns' (open for 'r') (1 item)
    'df': 'file:///Users/ryan/c/tiledb/repros/exp/ms/RNA/uns/df' (unopened)>

### Original `adata.uns` DataFrame

In [4]:
uns_df0

,uns_col
0,g
1,h
2,i
3,j
4,k
5,l


### Mutated `adata.uns` DataFrame

In [5]:
uns_df1 = adata.uns['df']
uns_df1

,index,uns_col
soma_joinid,,
0,0,g
1,1,h
2,2,i
3,3,j
4,4,k
5,5,l


### Extracted `uns` DataFrame

In [6]:
uns_df2 = _extract_uns(uns)['df']
uns_df2

,soma_joinid,index,uns_col
0,0,0,g
1,1,1,h
2,2,2,i
3,3,3,j
4,4,4,k
5,5,5,l
